In [2]:
## importing the necessary libraries
import pandas as pd
import numpy as np


## Extract Function

In [5]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import io
import time
from typing import List


class ExtractData:
    def __init__(self,start_date:str, end_date:str):

      self.start_date = datetime.strptime(start_date, "%Y-%m-%d")
      self.end_date = datetime.strptime(end_date, "%Y-%m-%d")

    def extract_nyc_yellow_taxi_data(self,
                 url:str ="https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page",):
      try:

        # Send GET request
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Regex pattern to match High Volume FHV files with date
        pattern = re.compile(r"(yellow_tripdata_)(\d{4}-\d{2})\.parquet", re.IGNORECASE)

        # Loop through all links
        download_links = []
        for link in soup.find_all("a", href=True):
            href = link["href"]
            match = pattern.search(href)
            if match:
                date_str = match.group(2)
                file_date = datetime.strptime(date_str, "%Y-%m")
                if file_date >= self.start_date and file_date <= self.end_date:
                    full_url = href if href.startswith("http") else f"https://www.nyc.gov{href}"
                    download_links.append((date_str, full_url))

        # Download and load each file into a DataFrame
        yellow_taxi_dfs = pd.DataFrame()
        for date_str, link in download_links:
            print(f"Downloading {date_str} from {link}")
            response = requests.get(link)
            if response.status_code == 200:
                # Convert bytes to file-like object
                buffer = io.BytesIO(response.content)
                df = pd.read_parquet(buffer)

                #adding to dataframe
                if yellow_taxi_dfs.empty:
                    yellow_taxi_dfs = df
                else:
                  pd.concat([yellow_taxi_dfs,df])

            else:
                print(f"Failed to download {link}")
        print(f"Data downloaded successfully, {yellow_taxi_dfs.shape}")
        print('\n')
        return yellow_taxi_dfs

      except Exception as e:
        return (f"Error: {e}")

    def extract_nyc_network_data(self, API_KEY: List,
                                location: str = "New York NY United States",
                                base_url: str = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"):

        start_date = self.start_date
        end_date = self.end_date

        current_date = start_date
        full_json = {
            "queryCost": 0,
            "latitude": None,
            "longitude": None,
            "resolvedAddress": None,
            "address": location.lower().replace(" ", ""),
            "timezone": None,
            "tzoffset": None,
            "description": None,
            "days": [],
            "alerts": [],
            "stations": {},
            "currentConditions": {}
        }

        while current_date <= end_date:
            next_month = (current_date.replace(day=28) + timedelta(days=4)).replace(day=1)
            month_end = min(next_month - timedelta(days=1), end_date)

            start_date_str = current_date.strftime("%Y-%m-%d")
            end_date_str = month_end.strftime("%Y-%m-%d")

            params = {
                "unitGroup": "us",
                "key": API_KEY,
                "include": "days,hours,current,alerts,stations",
                "contentType": "json"
            }

            url = f"{base_url}/{location}/{start_date_str}/{end_date_str}"
            print(f"Fetching data from {start_date_str} to {end_date_str}...")

            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()

                # Append daily data
                full_json["days"].extend(data.get("days", []))

                 # Extract hourly data nested within each day
                hourly_records = []
                fields = ['datetime', 'temp', 'dew', 'humidity', 'precip', 'preciptype', 'snow', 'snowdepth', 'visibility']
                for day in full_json.get("days", []):
                    for hour in day.get("hours", []):
                      filtered_hour = {key: hour.get(key) for key in fields}
                      filtered_hour["day"] = day.get("datetime")  # Add parent day for context
                      hourly_records.append(filtered_hour)

                ## create a pandas dataframe
                df_hours = pd.DataFrame(hourly_records)

                print(f"Retrieved {len(data.get('days', []))} days.")
            elif response.status_code == 429:
                print(f" Rate limit hit ({response.status_code}): {response.text}")
            else:
                print(f"Error {response.status_code}: {response.text}")

            current_date = next_month
            time.sleep(3)

        print(f"\n Retrieved data contain {df_hours.shape[0]//24} days of data.")
        return df_hours


## Transform Function

In [6]:
class DataTransformation:
    def __init__(self,nyc_taxi_data:pd.DataFrame, nyc_weather_data:pd.DataFrame):
        self.nyc_taxi_data = nyc_taxi_data
        self.nyc_weather_data = nyc_weather_data

    def transform_nyc_yellow_taxi_data(self):
      nyc_taxi_data = self.nyc_taxi_data

      # Converting the VendorID
      vendor_id= {1: "Creative Mobile Technologies, LLC",
                  2: "Curb Mobility, LLC",
                  6: 'Myle Technologies Inc',
                  7: 'Helix'
               }
      nyc_taxi_data['VendorID'] = nyc_taxi_data['VendorID'].map(vendor_id)

      # Handling the RateCard
      ratecode= {1: "Standard rate",
                 2: "JFK",
                 3: "Newark",
                 4: "Nassau or Westchester",
                 5: "Negotiated fare",
                 6: "Group ride",
                 99: 'Null/unknown'
              }
      nyc_taxi_data['RatecodeID'] = nyc_taxi_data['RatecodeID'].map(ratecode)

      ## Handling store and fwd flag
      store_and_forward= {
          'N': "No",
          'Y': "Yes"
      }
      nyc_taxi_data['store_and_fwd_flag'] = nyc_taxi_data['store_and_fwd_flag'].map(store_and_forward)

      ## Handling Payment type
      payment_type= {
          0: 'Flex Fair Trip',
          1: "Credit card",
          2: "Cash",
          3: "No charge",
          4: "Dispute",
          5: "Unknown",
          6: "Voided trip"
      }
      nyc_taxi_data['payment_type'] = nyc_taxi_data['payment_type'].map(payment_type)

      # Convert tpep_pickup_datetime and tpep_dropoff_datetime to datetime
      nyc_taxi_data['pickup_datetime'] = pd.to_datetime(nyc_taxi_data['tpep_pickup_datetime'])
      nyc_taxi_data['dropoff_datetime'] = pd.to_datetime(nyc_taxi_data['tpep_dropoff_datetime'])

      ## handling the Pulocation_Id and the DOLocation_Id
      lookup_table = pd.read_csv("https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv") ## importing the lookup table
      nyc_taxi_data = nyc_taxi_data.merge(lookup_table, left_on='PULocationID', right_on='LocationID', )
      nyc_taxi_data = nyc_taxi_data.merge(lookup_table, left_on='DOLocationID', right_on='LocationID')

      ## dropping the columns
      nyc_taxi_data.drop(columns=['PULocationID', 'DOLocationID'], inplace=True)
      nyc_taxi_data.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], inplace=True)
      nyc_taxi_data.drop(columns=['LocationID_x', 'LocationID_y'], inplace=True)

      ##renaming columns
      nyc_taxi_data.rename(columns={'Zone_x': 'PULocation_zone', 'Zone_y': 'DOLocation_zone'}, inplace=True)
      nyc_taxi_data.rename(columns={'Borough_x': 'PULocation_borough', 'Borough_y': 'DOLocation_borough'}, inplace=True)
      nyc_taxi_data.rename(columns={'service_zone_x': 'PULocation_service_zone', 'service_zone_y': 'DOLocation_service_zone'}, inplace=True)

      return nyc_taxi_data

    def transform_weather_data(self):
      nyc_weather_data = self.nyc_weather_data

      ## changing the datatype of datetime column to datetime
      nyc_weather_data['datetime'] = pd.to_datetime(nyc_weather_data['day'].astype(str) + ' ' + nyc_weather_data['datetime'].astype(str))
      nyc_weather_data.drop(columns=['day'], inplace=True)


      return nyc_weather_data


    def merge_data(self):
      taxi_data = self.transform_nyc_yellow_taxi_data()
      weather_data = self.transform_weather_data()

      ## merging the taxi data and the weather data together
      taxi_data['new_pickup_datetime'] = taxi_data['pickup_datetime'].dt.round('h')
      transformed_data = pd.merge(taxi_data, weather_data, left_on=['new_pickup_datetime'] , right_on=['datetime'], how='left')
      transformed_data.drop(columns=['new_pickup_datetime','datetime'], inplace=True)

      return transformed_data

    def transform(self):
      return self.merge_data()


## Load Pipeline

In [7]:
import os
import pandas as pd

class LoadData:
    def __init__(self, transformed_data: pd.DataFrame, file_name: str):
        self.transformed_data = transformed_data
        self.file_name = file_name

    def save(self):
        try:
            if not os.path.exists(self.file_name) or os.path.getsize(self.file_name) == 0:
                self.transformed_data.to_parquet(self.file_name, index=False)
                print('✅ Data saved successfully (new file or empty file)')
            else:
                existing_data = pd.read_parquet(self.file_name)
                combined_data = pd.concat([existing_data, self.transformed_data]).drop_duplicates()
                combined_data.to_parquet(self.file_name, index=False)
                print('✅ Data appended successfully')

            return self.transformed_data.shape

        except Exception as e:
            raise e

## ETL Pipeline

In [8]:
from typing import List

class ETLPipeline:
    def __init__(self, start_date: str, end_date: str, file_name: str):
        self.start_date = start_date
        self.end_date = end_date
        self.file_name = file_name

    def run(self, api_key:str):
        # Extract
        print('Extracting data...')

        extract = ExtractData(self.start_date, self.end_date)
        nyc_taxi_data = extract.extract_nyc_yellow_taxi_data()
        nyc_weather_data = extract.extract_nyc_network_data(api_key)

        #Transform
        transformer = DataTransformation(nyc_taxi_data, nyc_weather_data)
        transformed_data = transformer.transform()

        # Load
        load = LoadData(transformed_data, self.file_name)
        load.save()
        print(f'data saved to {self.file_name}')
        print(f'data shape: {load.save()}')
        print('\n')
        print('✅ ETL pipeline completed successfully')